# Kütüphaneler

In [2]:
import csv # CSV çıktısı verebilmek için
from itertools import combinations # Belirli boyuttta eleman kombinasyonları oluşturmak için

# Destek Değerini Hesaplayan Fonksiyon

In [3]:
def calculate_support(data, itemset):
    count = 0
    for transaction in data:
        if itemset.issubset(transaction):
            count += 1
    return count / len(data)

# Adayları Oluşturan Fonksiyon

In [4]:
def generate_candidates(frequent_itemsets, k):
    candidates = set()  # Boş aday kümesi
    frequent_items = list(frequent_itemsets)
    for i in range(len(frequent_items)):
        for j in range(i + 1, len(frequent_items)):
            union_set = frequent_items[i] | frequent_items[j]
            if len(union_set) == k:
                candidates.add(frozenset(union_set))
    return candidates

# Genel Algoritmayı Sağlayan Fonksiyon



In [6]:
def apriori(data, min_support):
    transactions = [set(transaction) for transaction in data]
    items = {frozenset([item]) for transaction in transactions for item in transaction}
    frequent_itemsets = {item for item in items if calculate_support(transactions, item) >= min_support}

    k = 2
    result = list(frequent_itemsets)
    while frequent_itemsets:
        candidates = generate_candidates(frequent_itemsets, k)
        frequent_itemsets = {itemset for itemset in candidates if calculate_support(transactions, itemset) >= min_support}
        result.extend(frequent_itemsets)
        k += 1
    return result

 # Kuralları Yazdıran Fonksiyon

In [9]:
def generate_rules(frequent_itemsets, data, min_confidence):
    transactions = [set(transaction) for transaction in data]
    rules = []
    for itemset in frequent_itemsets:
        subsets = [frozenset(x) for x in combinations(itemset, len(itemset) - 1)]
        for subset in subsets:
            remain = itemset - subset
            if remain:
                confidence = calculate_support(transactions, itemset) / calculate_support(transactions, subset)
                if confidence >= min_confidence:
                    rules.append((subset, remain, confidence))
    return rules


# CSV'den Okuyan Fonksiyon

In [10]:
def read_transactions_from_csv(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        return [row for row in reader]

# CSV'ye Yazdıran Fonksiyon

In [11]:
def write_results_to_csv(frequent_itemsets, rules, itemsets_file, rules_file):
    with open(itemsets_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Frequent Itemsets"])
        for itemset in frequent_itemsets:
            writer.writerow([", ".join(itemset)])

    with open(rules_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Antecedent", "Consequent", "Confidence"])
        for rule in rules:
            writer.writerow([", ".join(rule[0]), ", ".join(rule[1]), f"{rule[2]:.2f}"])

# Geri Kalanlar

In [20]:
# Örnek kullanım
input_csv = '/content/Transaction.csv'
output_itemsets_csv = 'frequent_itemsets.csv'
output_rules_csv = 'association_rules.csv'

data = read_transactions_from_csv(input_csv)

min_support = 0.5
min_confidence = 0.7

frequent_itemsets = apriori(data, min_support)

print("Sık Öğeler:")
for itemset in frequent_itemsets:
    print(itemset)

rules = generate_rules(frequent_itemsets, data, min_confidence)
print("\nKurallar:")
for rule in rules:
    print(f"{set(rule[0])} -> {set(rule[1])} (Confidence: {rule[2]:.2f})")

write_results_to_csv(frequent_itemsets, rules, output_itemsets_csv, output_rules_csv)

print("Sonuçlar CSV dosyalarına yazıldı.")

Sık Öğeler:
frozenset({'makarna'})
frozenset({'şeker'})
frozenset({'peynir'})
frozenset({'ekmek'})
frozenset({'peynir', 'ekmek'})
frozenset({'peynir', 'makarna'})
frozenset({'makarna', 'ekmek'})
frozenset({'peynir', 'makarna', 'ekmek'})

Kurallar:
set() -> {'makarna'} (Confidence: 0.80)
set() -> {'peynir'} (Confidence: 0.80)
set() -> {'ekmek'} (Confidence: 0.80)
{'peynir'} -> {'ekmek'} (Confidence: 0.75)
{'ekmek'} -> {'peynir'} (Confidence: 0.75)
{'peynir'} -> {'makarna'} (Confidence: 1.00)
{'makarna'} -> {'peynir'} (Confidence: 1.00)
{'makarna'} -> {'ekmek'} (Confidence: 0.75)
{'ekmek'} -> {'makarna'} (Confidence: 0.75)
{'peynir', 'makarna'} -> {'ekmek'} (Confidence: 0.75)
{'peynir', 'ekmek'} -> {'makarna'} (Confidence: 1.00)
{'makarna', 'ekmek'} -> {'peynir'} (Confidence: 1.00)
Sonuçlar CSV dosyalarına yazıldı.


In [19]:
# Öneri sistemi
def generate_recommendations(rules, user_input):
    recommendations = []
    for antecedent, consequent, confidence in rules:
        if user_input.issubset(antecedent):  # Kullanıcının girdisi antecedent'te varsa
            recommendations.append((consequent, confidence))
    recommendations.sort(key=lambda x: x[1], reverse=True)  # Güvene göre sırala
    return recommendations

user_input = input("Lütfen öneri için bir öğe giriniz (örnek: 'milk'): ").split(',')
user_set = set(user_input)
recommendations = generate_recommendations(rules, user_set)
print(recommendations)

Lütfen öneri için bir öğe giriniz (örnek: 'milk'): makarna
[(frozenset({'peynir'}), 1.0), (frozenset({'peynir'}), 1.0), (frozenset({'ekmek'}), 0.7499999999999999), (frozenset({'ekmek'}), 0.7499999999999999)]
